3. K-means klastersanje putem MapReduce programa (5 bodova)
Napisati map/reduce program koji vrši k-means klasterisanje liste vektora. Nasumično generisati podatke iz nekoliko normalnih raspodela u dve dimenzije i pokazati da algoritam radi (videti vežbe za primer). 

In [ ]:
from functools import reduce, partial
import multiprocessing, matplotlib.pyplot as plt, numpy as np, random, math
from sklearn.preprocessing import StandardScaler        # Needed for PCA.
from sklearn.decomposition import PCA

In [ ]:
def plot( fig, data, centroids, col_flag = False ):
    
    for i, d in enumerate( data ):
        col_indx = i // nk if col_flag is False else clusters[i]
        plt.scatter( d[1][0], d[1][1], color = colors[ col_indx ] )

    for i, cent in enumerate( centroids ):
        if cent is not None:
            plt.scatter( cent[0], cent[1], 500, marker = 'x', color = colors[i] )
    
    fig.show()

In [ ]:
nk = 100                        # Number of samples in the cluster.
k = 3                           # Number of clusters.
N = nk * k
n_dim = 2
mu, sigma = 500, 50             # Mean and standard deviation.
colors = [ 'r', 'g', 'b' ]
data = []

fig, ax = plt.subplots( figsize = ( 15, 10 ) )

indx = 0
for i in range( k ):
    rnd = random.randint( i * 250, ( i + 1 ) * 250 )

    for j in range( nk ):
        point = [ np.random.normal( rnd, sigma ) for i in range( n_dim ) ]
        data.append( ( indx, np.array( point ) ) )
        indx += 1
        
centroids = [ np.array( data[ random.randint( 0, N ) ][1] ) for i in range( k ) ]

plot( fig, data, centroids )


In [ ]:
def dist( x, y ):
    return math.sqrt( sum( ( p - q ) ** 2.0 for p, q in zip( x, y ) ) )

def map_calc_dist( point ):
    return [ dist( point[1], centroids[i] ) for i in range( k ) ]

def reduce_find_closest( acc, x ):
    #return acc + [ x.index( min( x ) ) ]
    return acc + [ np.argmin( x ) ]

def reduce_test( acc, x ):
    return acc + x          # acc and x are numpy arrays.

def map_test( key, value ):
    acc = np.zeros( shape = ( n_dim ) )
    return reduce( reduce_test, value, acc ) / len( value )

parallel_dict = dict()

pool = multiprocessing.Pool( multiprocessing.cpu_count() )
clusters = None

for i in range( 5 ):

    # This is parallel.
    cent_dists = list( map( map_calc_dist, data ) )
    clusters = reduce( reduce_find_closest, cent_dists, [] )

    # Preparing data for parallel processing.
    for i in range( k ):
        parallel_dict[i] = list()

    for i in range( N ):
        cluster_number = clusters[i]
        parallel_dict[ cluster_number ].append( data[i][1] )

    centroids = list( pool.starmap( map_test, parallel_dict.items() ) )
    print( centroids )

pool.terminate()

fig, ax = plt.subplots( figsize = ( 20, 10 ) )

plot( fig, data, centroids, col_flag = True )

In [ ]:
final_dict = dict()

for i in range( k ):
        final_dict[i] = list()

for i in range( N ):
    final_dict[ clusters[i] ].append( i )

for key, value in final_dict.items():
    print( key, ' -> ', value )